# Tutorial: Overview of Quantum Serverless

Structure:

- background, motivation and goals
- building blocks of distributed computation
- resource allocation and execution management
- assembling all together
- running as async program

## Background, motivation and goals

**Background and motivation**

Today a lot of experiments are happening on a local machines or on manually allocated remote resources. This approach is not scalable and we need to give users a tool to run quantum-classical workloads without worrying about allocating and managing underlying infrastructure.

To achieve the quantum advantage we need to figure out mechanism to combine Classical + Quantum computation, and how orchestrate it.


**Goals**

Create a set of tools that allow user to execute hybrid workloads (combine/orchestrate Classical and Quantum computation).

Efficiently manage abstractions for Quantum and Classical workloads.

Maximizes flexibility for users in multi-cloud and HPC scenarios.

## Building blocks of distributed computation

**Compute resources**

Compute resources can be described as set of computational nodes with resources associated to each of them.
Nodes are performing computation itself. 
Distributed storage is for accessing sharable resources between nodes.

![compute resources](https://raw.githubusercontent.com/Qiskit-Extensions/quantum-serverless/main/docs/tutorials/images/diagrams_compute_resource.png)


In order to resolve distributed compute we need a way to orchestrate our workloads on remote resources.

**Object**

Let's first look how to turn a local object into a remote one.

To turn your local object to distributed one you need to call `put` function. 
It will serialize your object and send it to distributed storage and return you a reference to remote object location. 

![objects diagram](https://raw.githubusercontent.com/Qiskit-Extensions/quantum-serverless/main/docs/tutorials/images/diagrams_put.png)

Example

```python
cirucit = QuantumCircuit(N, M)
...
circuit_reference = quantum_serverless.put(circuit)
```

Now we know how to move local object to remote compute resource. 
Next we need to do something with it. For this we will have remote functions.

**Functions**

As a second step we need to know how to transform our local function into remote one. 

In order to do that you need to annotate function with `distribute_task` decorator. 
This will turn your function into remote executable.  

![functions diagram](https://raw.githubusercontent.com/Qiskit-Extensions/quantum-serverless/main/docs/tutorials/images/diagrams_function.png)

All you need to do now is call this function and it will be executed as a remote procedure automatically. 

If you run N instances of this function in parallel you need just call it N times in a loop. 
The result of the following code will be a list of references to execution results.

Example

```python
@quantum_serverless.distribute_task()
def exp_val_remote(cirucit, obs):
    estimator = Estimator(...)
    return estimator.run(circuit, obs)

obs: Operator = ...

exp_val_execution_reference = exp_val_remote(circuit, obs)

circuit_list: List[QuantumCircuit] = [...]

obs_ref = quantum_serverless.put(obs)

exp_val_execution_references = [
    exp_val_remote(circ, obs_ref) 
    for circ in circuit_list
]
```

As you can see we are passing an observable by refernce and circuits by values. Attributes passed by value will be converted to remote objects automatically and resolved within the remote function. We are passing observable by referece because we know it is the same one shared by all parallel function invocations, so we are saving some space. 

**Collecting results**

And of course you need to collect your results back from remote resources. 
In order to do that you need to call the `get` function, which can be applied to object and function references or, alternatively, to a list of references.

![collecting results diagram](https://raw.githubusercontent.com/Qiskit-Extensions/quantum-serverless/main/docs/tutorials/images/diagram_get.png)

Example

```python
collected_circuit = quantum_serverless.get(circuit_reference)
# <QuantumCircuit ...>

collected_exp_value = quantum_serverless.get(exp_val_execution_reference)
# <EstimatorResult ...>
collected_exp_values = quantum_serverless.get(exp_val_execution_references)
# [<EstimatorResult ...>, <EstimatorResult ...>, ...]
```

## Resource allocation and execution management

**Resource allocation**

Some functions are more demanding than others. 
To allocate specific resources to a function, the decorator accepts `target` parameter.
For example, your function may require 2 cpus to be executed. You can pass `target={'cpu': 2}` to decorator and this function will request 2 cpus from your resource capacity to be executed.

![resource allocation diagram](https://raw.githubusercontent.com/Qiskit-Extensions/quantum-serverless/main/docs/tutorials/images/diagrams_resource_allocation.png)

Example

```python
@quantum_serverless.distribute_task(target={"cpu": 2, "mem": 8})
def exp_val_remote(cirucit, obs):
    estimator = Estimator(...)
    exp_val_result = estimator.run(circuit, obs)
    ...
    return result
```

**Execution management**

Now we figured how to convert our function and objects into remote counterparts and how to request for specific resources.
Next step would be deciding where to run your workloads. 

To have control over location of compute we use configurations in `QuantumServerless` object and python context managers. 

`QuantumServerless` has a notion of `providers` which are abstractions to define where your compute resources are located.

By default you always have a `local` provider which is your local machine where nodes and local cores.
And you can configure as many providers as you want.

![execution management diagram](https://raw.githubusercontent.com/Qiskit-Extensions/quantum-serverless/main/docs/tutorials/images/diagrams_context_management.png)

Example

```python
serverless = QuantumServerless({"providers": [...]})
serverless = QuantumServerless.load_configuration("<PATH_TO_CONFIG_FILE>")
print(serverless)
# <QuantumServerless: providers [local, ibm-cloud, ...]>

with serverless.context("local"):
    exp_val_execution_reference = exp_val_remote(circuit, obs)
    collected_exp_value = quantum_serverless.get(exp_val_execution_reference)

with serverless.context("ibm-cloud"):
    exp_val_execution_reference = exp_val_remote(circuit, obs)
    collected_exp_value = quantum_serverless.get(exp_val_execution_reference)
```

## Assembling all together

Now we have all the important concepts in place and we can assemble our first simple example of a distributed program.

> NOTE: make sure to run docker-compose to use this example

In [ ]:
from qiskit.circuit.random import random_circuit
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from quantum_serverless import QuantumServerless, distribute_task, put, get

serverless = QuantumServerless()

@distribute_task()
def exp_val_remote(circuit, obs):
    estimator = Estimator()
    return estimator.run(circuit, obs).result()

circuit_list = [random_circuit(2, 2) for _ in range(3)]
obs = SparsePauliOp(["IZ"])

with serverless.context():
    obs_ref = put(obs)
    exp_val_execution_references = [
        exp_val_remote(circ, obs_ref) 
        for circ in circuit_list
    ]
    collected_exp_values = get(exp_val_execution_references)
    
print(collected_exp_values)

## Running as async program

In most of the cases we want to run our scripts as sync programs, so we can lunch them and forget, then later on check results.
In order to do so we will use `Program` interface.

Let's reuse benchmark script which covers all the concepts covered above and which you can find [here](./source_files/benchmark.py).

In [3]:
from quantum_serverless import Program, GatewayProvider

gateway_provider = GatewayProvider(
    username="user",
    password="password123",
    host="http://gateway:8000",
)

program = Program(
    title="brnchmark_program",
    entrypoint="benchmark.py",
    working_dir="./source_files/",
    description="Benchmark program"
)

job = serverless.set_provider(gateway_provider).run_program(program)
job

<Job | 14>

In [4]:
job.status()

'SUCCEEDED'

In [6]:
print(job.logs())

Execution time: 9.168829679489136
Results: [[EstimatorResult(values=array([-4.04804878e-01+6.73804871e-01j, -1.94289029e-16-8.76563346e-01j,
        0.00000000e+00+2.93954499e-16j,  0.00000000e+00+1.00000000e+00j,
       -5.78736907e-01-2.02739106e-01j, -8.32667268e-17-1.00000000e+00j,
       -4.89359541e-16-1.15784406e-16j,  0.00000000e+00+3.87904546e-16j,
       -5.34438212e-01+5.55111512e-17j, -3.51585782e-17+1.06106206e+00j,
       -2.49262374e-02-2.61221914e-16j,  1.00000000e+00+1.00000000e+00j,
       -4.49088083e-01+0.00000000e+00j, -1.00000000e+00+5.73423770e-17j,
       -1.00000000e+00+4.90611306e-01j, -3.14018492e-16+9.81307787e-17j,
        1.81756717e-01-3.15451888e-02j, -1.33434861e+00-1.11022302e-16j,
        6.02950130e-02+1.56597418e-02j, -4.34512241e-01-3.34194313e-01j,
        0.00000000e+00+1.00000000e+00j,  0.00000000e+00-1.00000000e+00j,
        9.56512057e-01-2.30245304e-01j, -6.87854373e-01-7.04747198e-01j,
       -8.67447001e-01-2.58061563e-16j, -1.74208517e-16+